In [6]:
import os
import pathlib

import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

ROOT = pathlib.Path("D:\\ML_Dataset")
PATH = pathlib.Path(ROOT / "processed" / "training")
TEST_PATH = pathlib.Path("D:\\ML_Dataset\\bite_size\\test")
SAMPLE_SIZE = 50

In [7]:
def preprocess_image(image_path):
    # Load image with target size matching training input
    image = load_img(image_path, target_size=(150, 150))

    # Convert image to array
    image_array = img_to_array(image)

    # Expand dimensions to fit model input shape (batch size, height, width, channels)
    image_array = np.expand_dims(image_array, axis=0)

    # Normalize the image array as done during model training
    image_array /= 255.0

    return image_array

In [8]:
def predict_image_class(image_path):
    processed_image = preprocess_image(image_path)

    # Predict the class of the image
    prediction = model.predict(processed_image)

    # Assuming the model is binary for simplicity, adjust as needed for multiclass
    if prediction[0] > 0.5:
        return 1
    else:
        return 0

In [9]:
for i, path in enumerate(pathlib.Path(PATH).iterdir()):
    if i == SAMPLE_SIZE:
        break
    print(path.name)

actinotrocha
amphipods
appendicularia
appendicularia_house
bipinnaria
blurry
copepods
diatoms
echinodermata
eggs
larvae
malacostraca
medusae
mnemiopsis
noctiluca
phaeocystis
pilidium
pluteus
polychaeta
pteropods
rod
snow
unknown
worms


In [10]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(PATH,
                                                    target_size=(150, 150),
                                                    batch_size=SAMPLE_SIZE*5,
                                                    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(PATH,
                                                              target_size=(150, 150),
                                                              batch_size=SAMPLE_SIZE*5,
                                                              class_mode='binary')

Found 124114 images belonging to 24 classes.
Found 124114 images belonging to 24 classes.


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])


C:\Users\filip\PycharmProjects\ML-Project\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1463691575305718188
xla_global_id: -1
]


In [14]:
tf.config.list_physical_devices('GPU')

[]

In [15]:
tf.test.is_built_with_cuda()

False

In [ ]:
with tf.device('/GPU:0'):
    model.fit(train_generator,
              steps_per_epoch=20,
              epochs=5,
              validation_data=validation_generator,
              validation_steps=len(validation_generator))

C:\Users\filip\PycharmProjects\ML-Project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0018 - loss: -15633.4268

C:\Users\filip\PycharmProjects\ML-Project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [71]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test accuracy: {test_accuracy:.2f}")

2483/2483 ━━━━━━━━━━━━━━━━━━━━ 178s 72ms/step - accuracy: 0.0024 - loss: -4791260298169345376256.0000
Test accuracy: 0.00


In [72]:
for pic in TEST_PATH.iterdir():
    print(f"{pic.name}: {predict_image_class(pic)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
fs534_copepods_roi1.4031192200.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs534_copepods_roi1.4048441100.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs534_copepods_roi1.4133164900.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs534_copepods_roi1.4183715200.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
training_diatoms_roi0.4070788301.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
training_diatoms_roi0.4073816201.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
training_diatoms_roi2.4413740400.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
training_diatoms_roi2.4620018000.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
training_diatoms_roi3.4488066800.tif.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
training_diatoms_roi3.4503847800.tif.png: 1
